##### Negative binomial regression

Negative binomial regression is a type of generalized linear model in which the dependent variable is a count of the number of times an event occurs. In this case, the target variable is a count of dengue cases reported in each week.
And the data has high variance compared to mean, and dispersed data, so this regression model is matching for our data set.

## Load packages

In [1]:
## Load packages
import numpy as np    # fundamental package for scientific computing
import pandas as pd   # Python Data Analysis Library
import seaborn as sns # library for making statistical graphics in Python
import os             # operating system dependent functionality, file descriptor..

import statsmodels.api as sm                                     # Poission and Negative binomial regression
import statsmodels.formula.api as smf                            # statistical models formula APIs

from sklearn.metrics import mean_absolute_error                  # Mean absolute error regression loss
from sklearn.model_selection import train_test_split             # Split arrays or matrices into random train and test subsets

import warnings
warnings.filterwarnings('ignore')
debug_test = True

In [2]:
# Let's check what data files are available.
PATH="./../datasets/"
os.listdir(PATH)

['dengue_test_iq.pkl',
 'dengue_test_sj.pkl',
 'dengue_train_iq.pkl',
 'dengue_train_sj.pkl',
 'test_iq_month_1.pkl',
 'test_iq_month_10.pkl',
 'test_iq_month_11.pkl',
 'test_iq_month_12.pkl',
 'test_iq_month_2.pkl',
 'test_iq_month_3.pkl',
 'test_iq_month_4.pkl',
 'test_iq_month_5.pkl',
 'test_iq_month_6.pkl',
 'test_iq_month_7.pkl',
 'test_iq_month_8.pkl',
 'test_iq_month_9.pkl',
 'test_sj_month_1.pkl',
 'test_sj_month_10.pkl',
 'test_sj_month_11.pkl',
 'test_sj_month_12.pkl',
 'test_sj_month_2.pkl',
 'test_sj_month_3.pkl',
 'test_sj_month_4.pkl',
 'test_sj_month_5.pkl',
 'test_sj_month_6.pkl',
 'test_sj_month_7.pkl',
 'test_sj_month_8.pkl',
 'test_sj_month_9.pkl',
 'train_iq_month_1.pkl',
 'train_iq_month_10.pkl',
 'train_iq_month_11.pkl',
 'train_iq_month_12.pkl',
 'train_iq_month_2.pkl',
 'train_iq_month_3.pkl',
 'train_iq_month_4.pkl',
 'train_iq_month_5.pkl',
 'train_iq_month_6.pkl',
 'train_iq_month_7.pkl',
 'train_iq_month_8.pkl',
 'train_iq_month_9.pkl',
 'train_sj_month_1.pk

In [3]:
# let's load the train and test data# let's load the train and test data
train_filename_sj = ( './../datasets/dengue_train_sj.pkl' )
train_filename_iq = ( './../datasets/dengue_train_iq.pkl' )
test_filename_sj = ( './../datasets/dengue_test_sj.pkl' )
test_filename_iq = ( './../datasets/dengue_test_iq.pkl' )

dengue_train_sj = pd.read_pickle( train_filename_sj )
dengue_train_iq = pd.read_pickle( train_filename_iq )
dengue_test_sj = pd.read_pickle( test_filename_sj )
dengue_test_iq = pd.read_pickle( test_filename_iq )

In [4]:
#  RFE selected features from Linear Regression
col_RFE_sj = ['ndvi_ne', 'ndvi_nw', 'ndvi_se', 'ndvi_sw', 'reanalysis_air_temp_c',
       'reanalysis_avg_temp_c', 'reanalysis_dew_point_temp_c',
       'reanalysis_relative_humidity_percent',
       'reanalysis_specific_humidity_g_per_kg', 'ndvi_mean',
       'station_avg_temp_c_lagVar', 'reanalysis_max_air_temp_c_lagSum',
       'reanalysis_air_temp_c_lagSum', 'reanalysis_air_temp_c_lagMean',
       'reanalysis_air_temp_c_lagVar', 'reanalysis_avg_temp_c_lagSum',
       'reanalysis_avg_temp_c_lagMean', 'reanalysis_avg_temp_c_lagVar',
       'ndvi_ne_lagSum', 'ndvi_ne_lagMean', 'ndvi_ne_lagVar',
       'reanalysis_tdtr_c_lagSum', 'reanalysis_tdtr_c_lagVar',
       'ndvi_mean_lagVar', 'ndvi_nw_lagSum', 'ndvi_nw_lagVar',
       'ndvi_se_lagSum', 'ndvi_se_lagMean', 'ndvi_se_lagVar', 'ndvi_sw_lagSum',
       'ndvi_sw_lagMean', 'ndvi_sw_lagVar']
col_RFE_iq = ['ndvi_nw', 'ndvi_sw', 'reanalysis_dew_point_temp_c',
       'reanalysis_specific_humidity_g_per_kg',
       'reanalysis_specific_humidity_g_per_kg_lagVar',
       'reanalysis_dew_point_temp_c_lagVar', 'ndvi_se_lagVar',
       'ndvi_sw_lagVar', 'ndvi_ne_lagVar', 'ndvi_nw_lagVar']

In [5]:
# Through loop, we identified the optimum alpha value
#for alpha in range( 118, 130 ):
#    print( "Columns used are : ", alpha/100 )
# First we tried with the high correlated features, but the RFE features gave good result.
#col_full_sj = abs(dengue_train_sj.corr()).total_cases.drop(['total_cases', 'year']).sort_values(ascending = False).index[0:43]
col_full_sj = col_RFE_sj
X_sj = pd.DataFrame( dengue_train_sj, columns = col_full_sj)
y_sj = dengue_train_sj.total_cases
X_sj_Full_train, X_sj_Full_test, Y_sj_Full_train, Y_sj_Full_test = train_test_split( X_sj, y_sj, shuffle = False)

X_dengue_test_sj_Full = pd.DataFrame( dengue_test_sj, columns = col_full_sj)

In [6]:
# Formula
formula = ' + '.join([ str(feature) for feature in list(X_sj_Full_train.columns)])
formula = 'y ~ ' + formula

# Data formatting for San Juan city
sj_train      = X_sj_Full_train.copy()
sj_train['y'] = Y_sj_Full_train
sj_test       = X_sj_Full_test.copy()

nb_model = smf.glm( formula = formula,
                    data = sj_train,
                    family = sm.families.NegativeBinomial(alpha=1.18)).fit()
nb_model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                 Generalized Linear Model Regression Results                  
==============================================================================
Dep. Variable:                      y   No. Observations:                  699
Model:                            GLM   Df Residuals:                      672
Model Family:        NegativeBinomial   Df Model:                           26
Link Function:                    log   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -3131.7
Date:                Fri, 28 Jun 2019   Deviance:                       441.70
Time:                        23:11:02   Pearson chi2:                     484.
No. Iterations:                    20   Covariance Type:             nonrobust
=========================================================================================================
                                            coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------------------------------
Intercept                                15.6105     43.851      0.356      0.722     -70.335     101.556
ndvi_ne                                  -0.3140      0.723     -0.434      0.664      -1.732       1.104
ndvi_nw                                   0.0173      0.860      0.020      0.984      -1.668       1.702
ndvi_se                                  -0.4193      1.655     -0.253      0.800      -3.663       2.825
ndvi_sw                                  -0.1890      1.672     -0.113      0.910      -3.465       3.087
reanalysis_air_temp_c                    -2.9538      2.251     -1.312      0.189      -7.365       1.458
reanalysis_avg_temp_c                     1.4064      0.677      2.078      0.038       0.080       2.733
reanalysis_dew_point_temp_c               1.9299      1.977      0.976      0.329      -1.944       5.804
reanalysis_relative_humidity_percent     -0.2805      0.447     -0.627      0.530      -1.157       0.596
reanalysis_specific_humidity_g_per_kg    -0.8378      0.624     -1.343      0.179      -2.060       0.385
ndvi_mean                                -0.2263      0.230     -0.983      0.326      -0.678       0.225
station_avg_temp_c_lagVar                -0.3473      0.190     -1.830      0.067      -0.719       0.025
reanalysis_max_air_temp_c_lagSum          0.3482      0.054      6.422      0.000       0.242       0.454
reanalysis_air_temp_c_lagSum              0.7405      0.270      2.746      0.006       0.212       1.269
reanalysis_air_temp_c_lagMean             0.1851      0.067      2.746      0.006       0.053       0.317
reanalysis_air_temp_c_lagVar              1.8225      1.201      1.518      0.129      -0.531       4.176
reanalysis_avg_temp_c_lagSum             -0.8604      0.273     -3.147      0.002      -1.396      -0.325
reanalysis_avg_temp_c_lagMean            -0.2151      0.068     -3.147      0.002      -0.349      -0.081
reanalysis_avg_temp_c_lagVar             -1.2097      1.119     -1.081      0.280      -3.403       0.984
ndvi_ne_lagSum                            0.6390      0.280      2.280      0.023       0.090       1.188
ndvi_ne_lagMean                           0.1598      0.070      2.280      0.023       0.022       0.297
ndvi_ne_lagVar                          -19.9401      8.231     -2.423      0.015     -36.073      -3.807
reanalysis_tdtr_c_lagSum                 -0.3776      0.048     -7.801      0.000      -0.472      -0.283
reanalysis_tdtr_c_lagVar                  0.4228      0.299      1.412      0.158      -0.164       1.009
ndvi_mean_lagVar                         99.5360     44.411      2.241      0.025      12.492     186.580
ndvi_nw_lagSum                           -0.5352      0.349     -1.533      0.125      -1.219       0.149
ndvi_nw_lagVar                          -12.9203     11.763     -1.098      0.272     -3

- intercept value is not a big number, its good for the model.
- coefficient values of features are good values.

In [7]:
Y_sj_Full_pred = nb_model.predict(sj_test).astype(int)
print ("San Juan Test  MAE error :", mean_absolute_error(Y_sj_Full_pred, Y_sj_Full_test))

pred_train_sj = nb_model.predict(sj_train).astype(int)
print ("San Juan Train MAE error :", mean_absolute_error(pred_train_sj, Y_sj_Full_train))

Y_sj_pred_test = pd.Series(map(int, map(round, nb_model.predict( X_dengue_test_sj_Full ))))

San Juan Test  MAE error : 19.273504273504273
San Juan Train MAE error : 24.984263233190273


Compared to high correlation features, the RFE features gave good result

In [8]:
# Through loop, we identified the optimum alpha value
#for alpha in range( 118, 130 ):
#    print( "Columns used are : ", alpha/100 )
# col_full_iq = abs(dengue_train_iq.corr()).total_cases.drop(['total_cases', 'year']).sort_values(ascending = False).index[0:43]
col_full_iq = col_RFE_iq
X = pd.DataFrame( dengue_train_iq, columns = col_full_iq)
y = dengue_train_iq.total_cases
X_train, X_test, Y_train, Y_test = train_test_split( X, y, shuffle = False)

# Formula
formula = ' + '.join([ str(feature) for feature in list(col_full_iq)])
formula = 'y ~ ' + formula

# Data formatting for San Juan city
iq_train      = X_train.copy()
iq_train['y'] = Y_train
iq_test       = X_test.copy()

nb_model = smf.glm( formula = formula,
                    data = iq_train,
                    family = sm.families.NegativeBinomial(alpha = 0.001 )).fit()
nb_model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                 Generalized Linear Model Regression Results                  
==============================================================================
Dep. Variable:                      y   No. Observations:                  387
Model:                            GLM   Df Residuals:                      376
Model Family:        NegativeBinomial   Df Model:                           10
Link Function:                    log   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -2035.9
Date:                Fri, 28 Jun 2019   Deviance:                       2972.2
Time:                        23:11:03   Pearson chi2:                 4.15e+03
No. Iterations:                     6   Covariance Type:             nonrobust
================================================================================================================
                                                   coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------------------------------------
Intercept                                        7.1440      1.467      4.871      0.000       4.269      10.019
ndvi_nw                                         -1.2739      0.404     -3.153      0.002      -2.066      -0.482
ndvi_sw                                          0.7265      0.374      1.941      0.052      -0.007       1.460
reanalysis_dew_point_temp_c                     -1.5079      0.242     -6.239      0.000      -1.982      -1.034
reanalysis_specific_humidity_g_per_kg            1.6746      0.233      7.175      0.000       1.217       2.132
reanalysis_specific_humidity_g_per_kg_lagVar     1.7215      0.189      9.123      0.000       1.352       2.091
reanalysis_dew_point_temp_c_lagVar              -1.6308      0.178     -9.172      0.000      -1.979      -1.282
ndvi_se_lagVar                                 -64.7910      8.230     -7.872      0.000     -80.922     -48.660
ndvi_sw_lagVar                                 -17.4375      7.371     -2.366      0.018     -31.884      -2.991
ndvi_ne_lagVar                                   2.9176      8.465      0.345      0.730     -13.673      19.508
ndvi_nw_lagVar                                  48.6805      7.038      6.917      0.000      34.887      62.474
================================================================================================================
"""

In [9]:
Y_iq_pred = nb_model.predict(iq_test).astype(int)
print ("Iquitos Test  MAE error :", mean_absolute_error(Y_iq_pred, Y_test))

pred_train_iq = nb_model.predict(iq_train).astype(int)
print ("Iquitos Train MAE error :", mean_absolute_error(pred_train_iq, Y_train))
Y_iq_pred_test = pd.Series(map(int, map(round, nb_model.predict( pd.DataFrame( dengue_test_iq, columns = col_full_iq) ))))

Iquitos Test  MAE error : 7.976744186046512
Iquitos Train MAE error : 5.3462532299741605


In [10]:
pred = list(Y_sj_Full_pred)  + list(Y_iq_pred)
y_test = list( Y_sj_Full_test ) + list ( Y_test )
print( "Total Test  MAE : ", mean_absolute_error( y_test, pred ))

pred = list(pred_train_sj)  + list(pred_train_iq)
y_test = list( Y_sj_Full_train ) + list ( Y_train )
print( "Total Train MAE : ", mean_absolute_error( y_test, pred ))

Total Test  MAE :  15.258953168044076
Total Train MAE :  17.986187845303867


#####  Negative Binomial Model Results
| Model | city data | MAE |
|:---|:---|:---|
| Negative Binomial Train | San Juan | 24.98 |
| Negative Binomial Train| Iquitos | 5.34 |
| Negative Binomial Test | San Juan | 19.27 |
| Negative Binomial Test | Iquitos | 7.97 |


###### Poisson Regression

If the data is dispersed, this model is not good for preddiction. Still the data is count type of distribution, so we are trying poisson model too.

In [11]:
# For poisson model, the high correlation features gave good result compared to RFE features
col_full_sj = abs(dengue_train_sj.corr()).total_cases.drop(['total_cases', 'year']).sort_values(ascending = False).index[0:43]
X_sj = pd.DataFrame( dengue_train_sj, columns = col_full_sj)
y_sj = dengue_train_sj.total_cases
X_sj_Full_train, X_sj_Full_test, Y_sj_Full_train, Y_sj_Full_test = train_test_split( X_sj, y_sj, shuffle = False)

X_dengue_test_sj_Full = pd.DataFrame( dengue_test_sj, columns = col_full_sj)
# Formula
formula = ' + '.join([ str(feature) for feature in list(col_full_sj)])
formula = 'y ~ ' + formula

# Data formatting for San Juan city
sj_train      = X_sj_Full_train.copy()
sj_train['y'] = Y_sj_Full_train
sj_test       = X_sj_Full_test.copy()

po_model = smf.glm( formula = formula,
                    data = sj_train,
                    family = sm.families.Poisson()).fit()
po_model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                 Generalized Linear Model Regression Results                  
==============================================================================
Dep. Variable:                      y   No. Observations:                  699
Model:                            GLM   Df Residuals:                      669
Model Family:                 Poisson   Df Model:                           29
Link Function:                    log   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -12246.
Date:                Fri, 28 Jun 2019   Deviance:                       21080.
Time:                        23:11:03   Pearson chi2:                 2.85e+04
No. Iterations:                     5   Covariance Type:             nonrobust
=================================================================================================================
                                                    coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------------------------------------
Intercept                                       -10.2653     13.640     -0.753      0.452     -36.998      16.468
quarter                                           0.3718      0.025     15.051      0.000       0.323       0.420
weekofyear                                        0.0181      0.001     13.811      0.000       0.016       0.021
month                                            -0.0993      0.009    -11.003      0.000      -0.117      -0.082
reanalysis_specific_humidity_g_per_kg_lagMean    -0.0785      0.012     -6.556      0.000      -0.102      -0.055
reanalysis_specific_humidity_g_per_kg_lagSum     -0.3138      0.048     -6.556      0.000      -0.408      -0.220
reanalysis_dew_point_temp_c_lagMean              -0.1708      0.045     -3.757      0.000      -0.260      -0.082
reanalysis_dew_point_temp_c_lagSum               -0.6832      0.182     -3.757      0.000      -1.040      -0.327
station_avg_temp_c_lagMean                       -0.0254      0.003     -9.842      0.000      -0.030      -0.020
station_avg_temp_c_lagSum                        -0.1014      0.010     -9.842      0.000      -0.122      -0.081
station_min_temp_c_lagMean                        0.0237      0.001     16.030      0.000       0.021       0.027
station_min_temp_c_lagSum                         0.0949      0.006     16.030      0.000       0.083       0.107
station_max_temp_c_lagMean                        0.0082      0.001      7.133      0.000       0.006       0.011
station_max_temp_c_lagSum                         0.0330      0.005      7.133      0.000       0.024       0.042
reanalysis_min_air_temp_c_lagSum                  0.2553      0.010     24.572      0.000       0.235       0.276
reanalysis_min_air_temp_c_lagMean                 0.0638      0.003     24.572      0.000       0.059       0.069
reanalysis_max_air_temp_c_lagSum                  0.3260      0.009     34.526      0.000       0.308       0.345
reanalysis_max_air_temp_c_lagMean                 0.0815      0.002     34.526      0.000       0.077       0.086
reanalysis_air_temp_c_lagSum                      1.6518      0.185      8.933      0.000       1.289       2.014
reanalysis_air_temp_c_lagMean                     0.4129      0.046      8.933      0.000       0.322       0.504
reanalysis_avg_temp_c_lagSum                     -1.1446      0.044    -26.243      0.000      -1.230      -1.059
reanalysis_avg_temp_c_lagMean                    -0.2862      0.011    -26.243      0.000      -0.308      -0.265
reanalysis_specific_humidity_g_per_kg             0.0446      0.150      0.297      0.767      -0.250       0.339
reanalysis_relative_humidity_percent_lagSum       0.1998      0.039      5.175      0.000       0.124       0.276
reanalysis_relative_humidity_percent_lagMean      0.0500      0.010      5.175      

In [12]:
Y_sj_Full_pred = po_model.predict(sj_test).astype(int)
print ("San Juan Test  MAE error :", mean_absolute_error(Y_sj_Full_pred, Y_sj_Full_test))

pred_train_sj = po_model.predict(sj_train).astype(int)
print ("San Juan Train MAE error :", mean_absolute_error(pred_train_sj, Y_sj_Full_train))
Y_sj_pred_test = pd.Series(map(int, map(round, po_model.predict( X_dengue_test_sj_Full ))))

San Juan Test  MAE error : 19.512820512820515
San Juan Train MAE error : 26.283261802575108


In [13]:
# col_full_iq = abs(dengue_train_iq.corr()).total_cases.drop(['total_cases', 'year']).sort_values(ascending = False).index[0:43]
col_full_iq = col_RFE_iq
X = pd.DataFrame( dengue_train_iq, columns = col_full_iq)
y = dengue_train_iq.total_cases
X_iq_train, X_iq_test, Y_iq_train, Y_iq_test = train_test_split( X, y, shuffle = False)

# Formula
formula = ' + '.join([ str(feature) for feature in list(col_full_iq)])
formula = 'y ~ ' + formula

# Data formatting for San Juan city
iq_train      = X_iq_train.copy()
iq_train['y'] = Y_iq_train
iq_test       = X_iq_test.copy()

poiss_model = smf.glm( formula = formula,
                    data = iq_train,
                    family = sm.families.Poisson()).fit()
poiss_model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                 Generalized Linear Model Regression Results                  
==============================================================================
Dep. Variable:                      y   No. Observations:                  387
Model:                            GLM   Df Residuals:                      376
Model Family:                 Poisson   Df Model:                           10
Link Function:                    log   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -2051.9
Date:                Fri, 28 Jun 2019   Deviance:                       3006.8
Time:                        23:11:04   Pearson chi2:                 4.18e+03
No. Iterations:                     5   Covariance Type:             nonrobust
================================================================================================================
                                                   coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------------------------------------
Intercept                                        7.1135      1.462      4.865      0.000       4.248       9.979
ndvi_nw                                         -1.2766      0.403     -3.172      0.002      -2.066      -0.488
ndvi_sw                                          0.7289      0.373      1.955      0.051      -0.002       1.460
reanalysis_dew_point_temp_c                     -1.5014      0.241     -6.233      0.000      -1.974      -1.029
reanalysis_specific_humidity_g_per_kg            1.6679      0.233      7.172      0.000       1.212       2.124
reanalysis_specific_humidity_g_per_kg_lagVar     1.7169      0.188      9.132      0.000       1.348       2.085
reanalysis_dew_point_temp_c_lagVar              -1.6266      0.177     -9.180      0.000      -1.974      -1.279
ndvi_se_lagVar                                 -64.7520      8.199     -7.897      0.000     -80.822     -48.682
ndvi_sw_lagVar                                 -17.5531      7.345     -2.390      0.017     -31.949      -3.157
ndvi_ne_lagVar                                   2.9712      8.431      0.352      0.725     -13.553      19.495
ndvi_nw_lagVar                                  48.6884      7.009      6.946      0.000      34.951      62.426
================================================================================================================
"""

In [14]:
Y_iq_pred = poiss_model.predict(iq_test).astype(int)
print ("Iquitos Test  MAE error :", mean_absolute_error(Y_iq_pred, Y_iq_test))

pred_train_iq = poiss_model.predict(iq_train).astype(int)
print ("Iquitos Train MAE error :", mean_absolute_error(pred_train_iq, Y_iq_train))
X_dengue_test_iq = pd.DataFrame( dengue_test_iq, columns = col_full_iq)
Y_iq_pred_test = pd.Series(map(int, map(round, poiss_model.predict( X_dengue_test_iq ))))

Iquitos Test  MAE error : 7.984496124031008
Iquitos Train MAE error : 5.351421188630491


In [15]:
pred = list(Y_sj_Full_pred)  + list(Y_iq_pred)
y_test = list( Y_sj_Full_test ) + list ( Y_iq_test )
print( "Total Test  MAE : ", mean_absolute_error( y_test, pred ))

pred = list(pred_train_sj)  + list(pred_train_iq)
y_test = list( Y_sj_Full_train ) + list ( Y_iq_train )
print( "Total Train MAE : ", mean_absolute_error( y_test, pred ))

Total Test  MAE :  15.415977961432507
Total Train MAE :  18.82412523020258


#####  Poisson Model Results
| Model | city data | MAE |
|:---|:---|:---|
| Poisson Model Train | San Juan | 26.28 |
| Poisson Model Train| Iquitos | 5.35 |
| Poisson Model Test | San Juan | 19.51 |
| Poisson Model Test | Iquitos | 7.98 |


Compared to Poisson, Negative Binomial model is good based on MAE. But still this model is good compared to linear regression models.